# Content list
1. Import libraries
2. Import data
3. Security implications
4. Analysis
5. Export data

# 01. Import libraries

In [2]:
# Import analysis and visualization libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# 02. Import data

In [3]:
# Create path
path = r'C:\Users\adrig\OneDrive\Documentos\CARRIERFOUNDRY\DATA IMMERSION\ACHIEVEMENT 4\Ex_4.2_Python\03-2025 Instacart Basket Analysis'

In [4]:
# Import data
ords_prods_all = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_all.pkl'))

In [5]:
ords_prods_all.head(3)

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_last_order,new_customer,product_id,add_to_cart_order,reordered,...,spending_flag,days_last_order_median,frequency_flag,gender,state,age,customer_since,n_dependants,fam_status,income
0,2539329,1,1,2,8,NaN,True,196,1,0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2019-02-17,3,married,40423
1,2539329,1,1,2,8,NaN,True,14084,2,0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2019-02-17,3,married,40423
2,2539329,1,1,2,8,NaN,True,12427,3,0,...,Low spender,20.5,Non-frequent customer,Female,Alabama,31,2019-02-17,3,married,40423


# 03. Security implications 

PII data had been addressed in previous task (Notebook 4.9). 
Name and surname columns from customers were removed. 

# 04. Analysis

## 04.1 Comparison of customers behavior in different geographic areas. 

In [6]:
# Check column state
ords_prods_all['state'].nunique()

51

In [7]:
ords_prods_all['state'].value_counts(dropna=False)

state
Pennsylvania            667082
California              659783
Rhode Island            656913
Georgia                 656389
New Mexico              654494
Arizona                 653964
North Carolina          651900
Oklahoma                651739
Alaska                  648495
Minnesota               647825
Massachusetts           646358
Wyoming                 644255
Virginia                641421
Missouri                640732
Texas                   640394
Colorado                639280
Maine                   638583
North Dakota            638491
Alabama                 638003
Kansas                  637538
Louisiana               637482
Delaware                637024
South Carolina          636754
Oregon                  636425
Arkansas                636144
Nevada                  636139
New York                635983
Montana                 635265
South Dakota            633772
Illinois                633024
Hawaii                  632901
Washington              632852
Mi

In [8]:
# Define the regions
northeast = ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']
midwest = ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']
south = ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']
west = ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']

# Assign region values using loc() and isin()
ords_prods_all.loc[ords_prods_all['state'].isin(northeast), 'region'] = 'Northeast'

In [9]:
ords_prods_all.loc[ords_prods_all['state'].isin(midwest), 'region'] = 'Midwest'

In [10]:
ords_prods_all.loc[ords_prods_all['state'].isin(south), 'region'] = 'South'

In [11]:
ords_prods_all.loc[ords_prods_all['state'].isin(west), 'region'] = 'West'

In [12]:
# Check new column
ords_prods_all['region'].value_counts()

region
South        10791885
West          8292913
Midwest       7597325
Northeast     5722736
Name: count, dtype: int64

In [13]:
# Crosstab to compare spending habits by region
crosstab_spending_region = pd.crosstab(ords_prods_all['spending_flag'], ords_prods_all['region'], dropna=False)
crosstab_spending_region

region,Midwest,Northeast,South,West
spending_flag,,,,
High spender,29265,18642,40579,31223
Low spender,7568060,5704094,10751306,8261690


All regions have a very small proportion of high spenders. 
The South region has the highest count of both high and low spender, suggesting that it has the larger customer base. 

In [14]:
# Crosstab to compare spending habits by loyalty status
crosstab_spending_loyal = pd.crosstab(ords_prods_all['spending_flag'], ords_prods_all['loyalty_flag'], dropna=False)
crosstab_spending_loyal

loyalty_flag,Loyal customer,New customer,Regular customer
spending_flag,,,
High spender,13896,58628,47185
Low spender,10270197,6185362,15829591


## 04.2 Exclusion flag

In [13]:
# Create an exclusion flag for low-activity customers
ords_prods_all.loc[ords_prods_all['max_order'] < 5, 'exclusion_flag'] = 'Yes'

In [14]:
ords_prods_all.loc[ords_prods_all['max_order'] >= 5, 'exclusion_flag'] = 'No'

In [15]:
# Check frequency of values 
ords_prods_all['exclusion_flag'].value_counts(dropna=False)

exclusion_flag
No     30964564
Yes     1440295
Name: count, dtype: int64

In [16]:
ords_prods_all[['user_id', 'max_order', 'exclusion_flag']].sort_values('max_order').head(30)

,user_id,max_order,exclusion_flag
25156874,159838,1,Yes
25156873,159838,1,Yes
25156872,159838,1,Yes
25156871,159838,1,Yes
25156870,159838,1,Yes
19636925,124615,2,Yes
14449668,91567,2,Yes
8819967,55869,2,Yes
8819966,55869,2,Yes
8819965,55869,2,Yes


In [17]:
ords_prods_all.groupby('exclusion_flag')['user_id'].nunique()

exclusion_flag
No     162631
Yes     43578
Name: user_id, dtype: int64

43,578 customers are low-activity customers

In [18]:
# Remove observations from low_activity customers by creating a new dataframe
ords_prods_all_ex = ords_prods_all.drop(ords_prods_all[ords_prods_all['exclusion_flag'] =='Yes'].index)

In [19]:
ords_prods_all_ex.shape

(30964564, 33)

In [20]:
ords_prods_all.shape

(32404859, 33)

In [21]:
len(ords_prods_all) - len(ords_prods_all_ex)

1440295

In [26]:
# Remove exclusion flag
ords_prods_all_ex = ords_prods_all_ex.drop(columns = ['exclusion_flag'])

In [27]:
ords_prods_all_ex.shape

(30964564, 32)

# 05 Export data

In [28]:
# Export new dataframe with exclusions
ords_prods_all_ex.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_all_exclusions.pkl'))